#### In this notebook, we have provided some of the sub-products of producing the catalog as well as code to produce figures 2, 3, 4.

Please note that this joint catalog was generated using OM10 and cosmoDC2. Full pipelines to download and utilize the full dataproducts are available at https://github.com/drphilmarshall/OM10 and on NERSC.

We produce data products and catalogs generated for this project under sub-directory `data`. 

#### Note: Data used for plots in this file is too large to host on GitHub. Download the data from Zenodo (linked in README) and unzip inside the data folder.

In [ ]:
# REQUIRED IMPORTS
# FILE MANAGEMENT
import h5py
import os
from IPython.utils import io

%load_ext autoreload
%autoreload 2

# DATA MANIPULATION
import numpy as np
import pandas as pd


# STATISTICS
from scipy.stats import norm, truncnorm, uniform, beta, multivariate_normal

# VISUALIZATION
import matplotlib
import matplotlib.pyplot as plt
from astropy.visualization import simple_norm
import matplotlib.colors as mpc
import corner
from matplotlib.patches import Patch
import astropy.visualization as asviz


SMALL_SIZE = 17
MEDIUM_SIZE = 20
BIGGER_SIZE = 30

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams["font.family"] = "DejaVu Serif"

%matplotlib inline

### Figure 2

In [ ]:
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u


cosmo = FlatLambdaCDM(H0=72, Om0=0.26) #assumptions taken from OM10

def ellipticity2phi_q(e1, e2):
    """Transforms complex ellipticity moduli in orientation angle and axis ratio.

    :param e1: eccentricity in x-direction
    :param e2: eccentricity in xy-direction
    :return: angle in radian, axis ratio (minor/major)
    """
    
    phi = np.arctan2(e2, e1) / 2
    c = np.sqrt(e1**2 + e2**2)
    c = np.minimum(c, 0.9999)
    q = (1 - c) / (1 + c)
    return phi, q

def phi_q2_ellipticity(phi, q):
    """Transforms orientation angle and axis ratio into complex ellipticity moduli e1,
    e2.

    :param phi: angle of orientation (in radian)
    :param q: axis ratio minor axis / major axis
    :return: eccentricities e1 and e2 in complex ellipticity moduli
    """
    e1 = (1.0 - q) / (1.0 + q) * np.cos(2 * phi)
    e2 = (1.0 - q) / (1.0 + q) * np.sin(2 * phi)
    return e1, e2

def get_kpc(size, redshift):
    d_A = cosmo.angular_diameter_distance(z=redshift).to(u.kpc)
    theta = size*u.arcsec.to(u.radian)
    
    distance_kpc = (theta * d_A) # unit is kpc only now
    return distance_kpc.value

def all_logged_col(df, colname):
    df[f'{colname}_log'] = np.log10(df[colname])
    return df

def assign_red_bins(redshift_bins, df, redshift_col):
    df['bin'] = np.zeros(len(df))
    diff = redshift_bins[1] - redshift_bins[0]
    for red in redshift_bins:
        filt_ind = df[(df[redshift_col] >= red) & (df[redshift_col] < (red+ diff))].index
        df['bin'][filt_ind] = f'{np.round(red, 2)} - {np.round(red + diff, 2)}'
    return df

def add_useful_columns(df, nbins=7, zmin=0, zmax=3, cols_log = ['size_true', 'size_true_kpc', 'bulge_to_total_ratio_i', 
                  'stellar_mass', 'stellar_mass_bulge', 'stellar_mass_disk', 'halo_mass'], add_red_bins=False):
    
    df['phi_disk'], df['q_disk'] = ellipticity2phi_q(df['ellipticity_1_disk_true'], df['ellipticity_2_disk_true'])
    df['phi_bulge'], df['q_bulge'], = ellipticity2phi_q(df['ellipticity_1_bulge_true'], df['ellipticity_2_bulge_true'])
    df['phi'], df['q'] = ellipticity2phi_q(df['ellipticity_1_true'], df['ellipticity_2_true'])
    df['size_true_kpc'] = df.apply(lambda x: get_kpc(size = x['size_true'], redshift=x['redshift']), axis=1)
    for i in cols_log:
        df = all_logged_col(df, i)
    if add_red_bins:
        redshift_bins = np.linspace(zmin, zmax, nbins)
        df = assign_red_bins(redshift_bins, df, 'redshift')
    return df
    

In [ ]:
### read in all data 
cosmodc2 = pd.read_csv('data/all_cosmo_data.csv', index_col=0)
cosmodc2 = add_useful_columns(cosmodc2)
final_AGN = pd.read_csv("data/final_AGN_below_z3.csv", index_col=0)
final_AGN = add_useful_columns(final_AGN)
all_om10 = pd.read_csv('data/original_om10.csv', index_col=0)
notAGN = pd.read_csv("data/COSMODC2_notAGN.csv", index_col=0)
notAGN = add_useful_columns(notAGN)

final_lens = pd.read_csv("data/final_lens_below_z3.csv", index_col=0)
final_lens = add_useful_columns(final_lens)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["font.family"] = "DejaVu Serif"

props3 = dict(boxstyle='round', facecolor='thistle', alpha=0.5)

fig, ax = plt.subplots(1, 3, figsize=(30, 11))
plt.subplots_adjust(wspace=0.4)  # spacing between panels

### select a couple points
subset_idx = np.random.choice(len(cosmodc2), size=min(5000, len(cosmodc2)), replace=False)
ax[0].scatter(cosmodc2['redshift'].iloc[subset_idx],
              cosmodc2['M_i'].iloc[subset_idx],
              s=30, alpha=0.2, color='gray',
              label='cosmoDC2 population')



ax[0].scatter(final_AGN['ZSRC'], final_AGN['ABMAGI_IN4'],
              s=40, edgecolor='k', alpha=0.9, color='crimson',
              label='OM10')

ax[0].scatter(final_AGN['redshift'], final_AGN['M_i'],
              s=40, alpha=0.9, edgecolor='k', color='royalblue',
              label='Matched cosmoDC2')

# Legend moved outside panel to the right
lgnd = ax[0].legend(
    loc='upper right',
    bbox_to_anchor=(1.01, 1.02),
    frameon=True
)
handles = lgnd.legend_handles

for handle in handles:
    handle.set_sizes([100])

ax[0].set_xlabel("Redshift", size=30)
ax[0].set_ylabel("Source $M_i$", size=30)
ax[0].set_title("Nearest neighbor mapping", size=30)
ax[0].minorticks_on()

# Panel 2: Redshift correlation
ax[1].scatter(final_AGN['ZSRC'], final_AGN['redshift'],
              s=40, edgecolor='k', color='black')
ax[1].plot(final_AGN['ZSRC'], final_AGN['ZSRC'],
           color='gold', ls='--', lw=3)

ax[1].text(0.05, 0.9,
           'Correlation: %.2f' % np.round(
               np.corrcoef(final_AGN['ZSRC'], final_AGN['redshift'])[1][0], 2),
           fontsize=25, transform=ax[1].transAxes, bbox=props3)

ax[1].set_xlabel("OM10 Redshift", size=30)
ax[1].set_ylabel("cosmoDC2 Redshift", size=30)
ax[1].set_title("Redshift", size=30)
ax[1].minorticks_on()

# Panel 3: Mi correlation, colored by redshift
sc3 = ax[2].scatter(final_AGN['ABMAGI_IN4'], final_AGN['M_i'],color='k',
                    s=40, edgecolor='k', alpha=0.9)

ax[2].plot(final_AGN['ABMAGI_IN4'], final_AGN['ABMAGI_IN4'],
           color='gold', ls='--', lw=3)

ax[2].text(0.05, 0.9,
           'Correlation: %.2f' % np.round(
               np.corrcoef(final_AGN['ABMAGI_IN4'], final_AGN['M_i'])[1][0], 2),
           fontsize=25, transform=ax[2].transAxes, bbox=props3)

ax[2].set_xlabel("OM10 $M_{i}$", size=30)
ax[2].set_ylabel("cosmoDC2 $M_{i}$", size=30)
ax[2].set_title("$M_{i}$", size=30)
ax[2].minorticks_on()

# Overall title

fig.suptitle("OM10–cosmoDC2 Mapping", fontsize=36, y=1.0)

# Tick size adjustments (major & minor)
for axis in ax:
    axis.tick_params(axis='both', which='major', length=10, width=2, labelsize=20)
    axis.tick_params(axis='both', which='minor', length=6, width=1.5)

fig.tight_layout()



### Figure 3

In [ ]:
from latils import make_contour

In [ ]:
def add_einstein(df):
    df['EINSTEIN'] = ((4 * np.pi * (df["VELDISP"] ** 2) * df["DDS"]) / ((3e5) ** 2 * df["DS"]))/(2 * np.pi / 360 / 3600)
    return df
all_om10 = add_einstein(all_om10)
all_om10['stellar_mass_log'] = np.array(notAGN.sample(len(all_om10))['stellar_mass_log'])
all_om10['size_true_kpc_log'] = np.array(notAGN.sample(len(all_om10))['size_true_kpc_log'])
all_om10['Mag_true_i_lsst_z0'] = np.array(notAGN.sample(len(all_om10))['Mag_true_i_lsst_z0'])
fig = make_contour([all_om10[['ZLENS', 'stellar_mass_log','size_true_kpc_log', 'Mag_true_i_lsst_z0']], 
               final_lens[['ZLENS','stellar_mass_log', 'size_true_kpc_log','ABMAG_I']]], 
             labels = ['$z_{lens}$','$\log(M_\star / M_\odot)$', '$\log(R_{eff} / kpc)$',"$M_{i, lens}$"], 
             categories = ['Population', 'Selection'],
             colors = ['darkslategray', 'firebrick'], 
             range_for_bin=True, show_correlation=True)
# fig.savefig('lens_selection.pdf')

In [ ]:

fig = make_contour([cosmodc2[['redshift','stellar_mass_log', 'size_true_kpc_log', 'Mag_true_i_lsst_z0']], 
               final_AGN[['ZSRC','stellar_mass_log', 'size_true_kpc_log','Mag_true_i_lsst_z0']]], 
             labels = ['$z_{src}$','$\log(M_\star / M_\odot)$', '$\log(R_{eff} / kpc)$',"$M_{i, src}$"], 
             categories = ['Population', 'Selection'],
             colors = ['darkslategray', 'dodgerblue'], 
             range_for_bin=True, show_correlation=True)
# fig.savefig('source_selection.pdf', facecolor='white');

### Figure 4

In [ ]:
### original source paths
paths = ['/pscratch/sd/v/vpadma/generated_images/test/0118/all',
         '/pscratch/sd/v/vpadma/generated_images/test/0118/nolens',
         '/pscratch/sd/v/vpadma/generated_images/test/0118/nosrc',
         '/pscratch/sd/v/vpadma/generated_images/test/0118/deconvolved/all',
         '/pscratch/sd/v/vpadma/generated_images/test/0118/deconvolved/nolens',
         '/pscratch/sd/v/vpadma/generated_images/test/0118/deconvolved/nosrc',]
### original source indices
image_ind = [525, 1284, 1100, 39, 900]


In [ ]:
# Read image_list from the HDF5 file
read_image_list = []
data_file = 'data/figure4_images.h5'
with h5py.File(data_file, "r") as h5f:
    for group_name in h5f.keys():
        group = h5f[group_name]
        image_set = [group[f"image_{i}"][:] for i in range(len(group))]
        read_image_list.append(image_set)

In [ ]:
from image_plotting_functions import plot_collage, obtain_collage

fig, ax = plt.subplots(3, 6, figsize=(24, 14))
fig.subplots_adjust(wspace=0, hspace=0)

# Plot images
for i in range(3):
    for j in range(5):
        norm = simple_norm(
            read_image_list[0][j],
            stretch='asinh',
            vmax=1.5 * np.max(read_image_list[0][j]),
            log_a=5000,
            asinh_a=0.01,
            sinh_a=0.5
        )

        ax[i, j+1].imshow(read_image_list[i][j], norm=norm)
        ax[i, j+1].axis('off')

    # First column
    ax[i, 0].imshow(read_image_list[i+3][0], norm=norm)
    ax[i, 0].axis('off')

# Column labels
col_labels = [
    "Deconvolved image A", "Image A", "Image B",
    "Image C", "Image D", "Image E"
]
for col, label in enumerate(col_labels):
    fig.text(
        (col + 0.5) / 6,  # x position in figure coords
        0.97,             # y position in figure coords
        label,
        ha='center', va='bottom', fontsize=25
    )

# Row labels (rotated)
row_labels = [
    "All light included",
    "Lens light subtracted",
    "Lens+AGN light subtracted"
]
for row, label in enumerate(row_labels):
    fig.text(
        0.0,                          # x position
        1 - (row + 0.5) / 3,           # y position
        label,
        ha='center', va='center',
        rotation='vertical',
        fontsize=25
    )

# Example vertical line
fig.add_artist(plt.Line2D(
    [1/6 + 0.004, 1/6 + 0.004],
    [0, 1],
    lw=4,
    transform=fig.transFigure,
    color="black"
))

fig.tight_layout()
# fig.savefig('figures/collage_test.pdf', bbox_inches='tight', dpi=600)
